# Initializing 

### Import/Set up

In [1]:
import time 

start = time.time()

In [2]:
import matplotlib 
matplotlib.use('Agg')

In [3]:
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 
import numpy as np
import seaborn as sns 

%matplotlib inline
#%config Completer.use_jedi = False 

In [4]:
from brian2 import *
import brian2genn

# outputDirectory = '/work/kbw29/brian2Genn/Output'
outputDirectory = '/hpc/home/kbw29/Code/brian2Genn/Output'


set_device('genn', directory = outputDirectory,compile=True, run=True, debug=True)

INFO       The following preferences have been changed for Brian2GeNN, reset them manually if you use a different device later in the same script: codegen.loop_invariant_optimisations, core.network.default_schedule [brian2.devices.genn]


#### Personal Functions 

In [5]:
def visualise_connectivity(S):
    Ns = len(S.source)
    Nt = len(S.target)
    figure(figsize=(10, 4))
    subplot(121)
    plot(zeros(Ns), arange(Ns), 'ok', ms=10)
    plot(ones(Nt), arange(Nt), 'ok', ms=10)
    for i, j in zip(S.i, S.j):
        plot([0, 1], [i, j], '-k')
    xticks([0, 1], ['Source', 'Target'])
    ylabel('Neuron index')
    xlim(-0.1, 1.1)
    ylim(-1, max(Ns, Nt))
    subplot(122)
    plot(S.i, S.j, 'ok')
    xlim(-1, Ns)
    ylim(-1, Nt)
    xlabel('Source neuron index')
    ylabel('Target neuron index')

In [6]:
def plot_spikes(monitor, stateMonitor, rangeNeurons):
    for i in range(rangeNeurons):
        spikes = (monitor.t[monitor.i == i] - defaultclock.dt)/ms
        val = stateMonitor[i].V
        subplot(rangeNeurons, 1, i+1)
        plot(tile(spikes, (2, 1)), vstack((val[array(spikes, dtype=int)], zeros(len(spikes)))), 'C0')
        title("Spikes")
    tight_layout()
    show()

In [7]:
def plot_potential(monitor, stateMonitor, rangeNeurons):
    for i in range(rangeNeurons):
        spikes = (monitor.t[monitor.i == i] - defaultclock.dt)/ms
        val = stateMonitor[i].V
        subplot(rangeNeurons, 1, i+1)
        plot(stateMonitor.t/ms, val)
        title("Trace")
    tight_layout()
    show()

In [8]:
def plot_population(prMonitorRun, spikeMonitorRun, labelGroup, nActivity, pRun): 
    # plotting
    title('Population rates')
    xlabel('ms')
    ylabel('Hz')

    plot(prMonitorRun.t / ms, prMonitorRun.smooth_rate(width=25 * ms) / Hz, label=labelGroup)

    legend()
    figure()

    title('Population activities ({} neurons/pop)'.format(nActivity))
    xlabel('ms')
    yticks([])

    plot(spikeMonitorRun.t / ms, spikeMonitorRun.i + (pRun + 1) * nActivity, '.', markersize=2, label=labelGroup) 


    legend()
    show()

In [9]:
def plot_population_Multi(prMonitorRun_smooth, prMonitorRun_t, spikeMonitorRun, labelGroup, nActivity, pRun): 
    # plotting
    title('Population rates')
    xlabel('ms')
    ylabel('Hz')

    plot(prMonitorRun_t / ms, prMonitorRun_smooth, label=labelGroup)

    legend()
    figure()

    title('Population activities ({} neurons/pop)'.format(nActivity))
    xlabel('ms')
    yticks([])

    plot(spikeMonitorRun.t / ms, spikeMonitorRun.i + (pRun + 1) * nActivity, '.', markersize=2, label=labelGroup) 


    legend()
    show()

In [10]:
def plot_population_Multi_2(prMonitorRun_smooth, prMonitorRun_t, labelGroup): 
    # plotting
    title('Population rates')
    xlabel('ms')
    ylabel('Hz')

    plot(prMonitorRun_t / ms, prMonitorRun_smooth, label=labelGroup)

    legend()
    show()

In [11]:
def active_percent(monitor, binSize, durationSimulation, numNeurons, ymax):
    # Define bin size 
    bin_size = binSize*ms
    # Define duration of simulation 
    duration = durationSimulation*second
    # Define number of neurons 
    num_Neurons = numNeurons

    # Initialize array to record spike occurancces 
    spkTotal = np.zeros([int(duration/bin_size)])
    
    for current in range(num_Neurons): 
        spk_count, bin_edges = np.histogram(a = np.r_[monitor.t[monitor.i == current]/second],
                                            bins = int(duration/bin_size), range = (0,durationSimulation))
        # Convert spike count to binary 
        binary = np.where(spk_count > 0, 1, 0)
        # Add to recording array
        spkTotal = spkTotal + binary

    plt.plot(bin_edges[:-1], (spkTotal/num_Neurons)*100)
    plt.axis([0, durationSimulation, 0, ymax])
    plt.title('Percent active granule cells as function of time')
    plt.ylabel('Active Cell (%)')
    plt.xlabel('Time (s)')
    plt.show()

In [12]:
def averaged_ActivePercent(spkMulti, spkSingle, binSize, durationSimulation, numNeurons, ymax):
    # Define bin size 
    bin_size = binSize*ms
    # Define duration of simulation 
    duration = durationSimulation*second
    # Define number of neurons 
    num_Neurons = numNeurons
    
    spk_count, bin_edges = np.histogram(a = np.r_[spkSingle.t[spkSingle.i == 1]/second],
                                        bins = int(duration/bin_size), range = (0,durationSimulation))

    averaged_spkMulti = np.mean(spkMulti, axis = 0)
    
    plt.plot(bin_edges[:-1], averaged_spkMulti)
    plt.axis([0, durationSimulation, 0, ymax])
    plt.title('Percent active granule cells as function of time')
    plt.ylabel('Active Cell (%)')
    plt.xlabel('Time (s)')
    plt.show()

#### General paramaters 

In [13]:
# spatial 
#gC = 320
gC = 10
numGC = gC**2 
gG = 32 
#numGolgi = gG**2  
numGolgi = 10 
gGlo = 31
numGlomeruli = gGlo**2

# Spatial paramaters 
gcSpacing = ((gG*(gG-1))/(gC-1))*umetre 
golgiSpacing = 32*umetre 

gcSpacing = ((gG*(gG-1))/(gC-1))*umetre 


numPKJ = numBS = 10

# Neuron Models

### Granule Cells 

In [14]:
# Parameters 

# neuron parameters
theta_GR = -35*mV # threshold (from Yamazaki) 
Cm_GR = 3.1*pF 

# Conductances 
gL_GR = 0.43*nS 
g_AMPA_GR = 0.18*nS 
g_NMDA_GR = 0.025*nS 
g_Inh_GR = 0.028*nS 
g_ahp_GR = 1*nS

# Various leaks (from Yamazaki) 
El_GR = -58*mV
Eexc = 0*mV 
E_inh_GR = -82*mV 
E_ahp_GR = -82*mV # after hyperpolarization 

# Tau
tau_ahp_GR = 5*ms 
tau_AMPA_GR = 1.2*ms 
tau_NMDA_GR = 52.0*ms 
tau_GABA_1_GR = 7*ms
tau_GABA_2_GR = 59*ms

eqs_GR = Equations('''
                        dV / dt = (-gL_GR * (V - El_GR) - I_syn) / Cm_GR : volt 
                        
                        I_syn = I_AMPA + I_NMDA + I_GABA_tot + I_ahp: amp 
                        I_GABA_tot = 0.43*I_GABA_1 + 0.57*I_GABA_2 : amp 
                        
                        I_AMPA = g_AMPA_GR * (V - Eexc) * s_AMPA : amp
                        ds_AMPA / dt = -s_AMPA / tau_AMPA_GR : 1 
                        
                        I_NMDA = g_NMDA_GR * (V - Eexc) * s_NMDA : amp
                        ds_NMDA / dt = -s_NMDA / tau_NMDA_GR : 1 
                        
                        I_GABA_1 = g_Inh_GR * (V - E_inh_GR) * s_GABA_1 : amp
                        ds_GABA_1 / dt = -s_GABA_1 / tau_GABA_1_GR : 1 
                        
                        I_GABA_2 = g_Inh_GR * (V - E_inh_GR) * s_GABA_2 : amp
                        ds_GABA_2 / dt = - s_GABA_2 / tau_GABA_2_GR : 1 
                        
                        I_ahp = g_ahp_GR * (V - E_ahp_GR) * s_ahp_GR: amp
                        ds_ahp_GR / dt = - s_ahp_GR / tau_ahp_GR : 1
                        
                        x : metre 
                        y : metre 
                ''')

# Defining granule cell neuron group 
granuleCells = NeuronGroup(N=numGC, model=eqs_GR, reset='V = El_GR',
                    threshold='V > theta_GR',
                    method='euler')
granuleCells.V = El_GR 

In [15]:
# Defining MF input 

# Defining input parameters 
num_inputs = 100
f = 0.5*Hz 
eq_rate = '(22.5 + 7.5*cos((2*pi*t*f) + pi))*Hz'

# Equations 
eqs_glut_MFGR = '''
                   w_MFGR : 1 
                '''
eqs_pre_glut_MFGR = '''
                       s_AMPA += w_MFGR
                    '''

P_MFGR = PoissonGroup(num_inputs, rates = eq_rate)
S_MFGR = Synapses(P_MFGR, granuleCells, model = eqs_glut_MFGR, on_pre = eqs_pre_glut_MFGR, method = 'euler') 
S_MFGR.connect(condition = 'i == j')
S_MFGR.w_MFGR = 4


In [16]:
# Visualize synaptic connections 
#visualise_connectivity(S_MFGR)

### Golgi

In [17]:
# Parameters 

# neuron parameters
theta_GO = -52*mV # threshold (from Yamazaki) 
Cm_GO = 28.0*pF 

# Conductances 
gL_GO = 2.3*nS 
g_AMPA_GO = 45.5*nS 
g_NMDA_GO = 30.0*nS 
g_ahp_GO = 20*nS

El_GO = -55*mV
E_exc = 0*mV 
E_ahp_GO = -72.7*mV # after hyperpolarization 

# Tau
tau_AHP_GO = 5*ms 
tau_AMPA_GO = 1.5*ms 
tau_NMDA_1_GO = 31.0*ms 
tau_NMDA_2_GO = 170.0*ms 

eqs_GO = Equations('''
                        dV / dt = (-gL_GO * (V - El_GO) - I_syn) / Cm_GO : volt 
                        
                        I_syn = I_AMPA + I_NMDA_tot + I_ahp : amp 
                        I_NMDA_tot = 0.33*I_NMDA_1 + 0.67*I_NMDA_2 : amp
                        
                        I_AMPA = g_AMPA_GR * (V - Eexc) * s_AMPA : amp
                        ds_AMPA / dt = -s_AMPA / tau_AMPA_GO : 1 
                        
                        I_NMDA_1 = g_NMDA_GO * (V - E_exc) * s_NMDA_1 : amp
                        ds_NMDA_1 / dt = -s_NMDA_1 / tau_NMDA_1_GO : 1 
                        
                        I_NMDA_2 = g_NMDA_GO * (V - Eexc) * s_NMDA_2 : amp
                        ds_NMDA_2 / dt = -s_NMDA_2 / tau_NMDA_2_GO : 1 
                        
                        I_ahp = g_ahp_GO * (V - E_ahp_GO) * s_ahp_GO : amp
                        ds_ahp_GO / dt = - s_ahp_GO / tau_AHP_GO : 1
                        
                        x : metre 
                        y : metre 
                ''')

# Defining golgi cell neuron group 
golgi = NeuronGroup(N=numGolgi, model=eqs_GO, reset='V = El_GO',
                    threshold='V > theta_GO',
                    method='euler')
golgi.V = El_GO 

### Purkinje

In [18]:
# Parameters 

# neuron parameters
theta_PKJ = -55*mV # threshold (from Yamazaki) 
Cm_PKJ = 107.0*pF 
I_spont_PKJ = 0.25*nA 

# Conductances 
gL_PKJ = 2.32*nS 
g_AMPA_PKJ = 0.7*nS 
g_GABA_PKJ = 1.0*nS
g_AHP_PKJ = 0.1*nS

# Leak potentials 
El_PKJ = -68.0*mV
E_exc = 0*mV 
E_inh_PKJ = -75.0*mV
E_AHP_PKJ = -70.0*mV # after hyperpolarization 

# Tau
tau_AMPA_PKJ = 8.3*ms 
tau_GABA_PKJ = 10*ms
tau_AHP_PKJ = 5*ms 

eqs_PKJ = Equations('''
                        dV / dt = (-gL_PKJ * (V - El_PKJ) - I_syn + I_spont_PKJ) / Cm_PKJ : volt 
                        
                        I_syn = I_AMPA + I_GABA + I_AHP : amp 
                        
                        I_AMPA = g_AMPA_PKJ * (V - E_exc) * s_AMPA : amp
                        ds_AMPA / dt = - s_AMPA / tau_AMPA_PKJ : 1 
                        
                        I_GABA = g_GABA_PKJ * (V - E_inh_PKJ) * s_GABA : amp 
                        ds_GABA / dt = - s_GABA / tau_GABA_PKJ : 1
                        
                        I_AHP = g_AHP_PKJ * (V - E_AHP_PKJ) * s_AHP_PKJ : amp
                        ds_AHP_PKJ / dt = - s_AHP_PKJ / tau_AHP_PKJ : 1
                    ''')

# Defining purkinje cell neuron group 
purkinje = NeuronGroup(N = numPKJ, model = eqs_PKJ, reset = 'V = El_PKJ',
                    threshold = 'V > theta_PKJ',
                    method = 'euler')
purkinje.V = El_PKJ

In [19]:
# Defining CF input 

# Defining input parameters 
num_inputs = numPKJ
f = 0.5*Hz 
eq_rate_CF = '(2.25 + 0.75*cos((2*pi*t*f) + pi))*Hz'

# Equations 
eqs_glut_CFPKJ = '''
                   w_CFPKJ : 1 
                 '''
eqs_pre_glut_CFPJK = '''
                       s_AMPA += w_CFPKJ
                    '''

P_CFPKJ = PoissonGroup(num_inputs, rates = eq_rate_CF)
S_CFPKJ = Synapses(P_CFPKJ, purkinje, model = eqs_glut_CFPKJ, on_pre = eqs_pre_glut_CFPJK, method = 'euler') 
S_CFPKJ.connect(condition = 'i == j')
S_CFPKJ.w_CFPKJ = 1.0

In [20]:
# Visualize synaptic connections 
#visualise_connectivity(S_CFPKJ)

### Basket Cells

In [21]:
# Parameters 

# neuron parameters
theta_BS = -55*mV # threshold (from Yamazaki) 
Cm_BS = 107.0*pF 

# Conductances 
gL_BS = 2.32*nS 
g_AMPA_BS = 0.7*nS 
g_AHP_BS = 0.1*nS

# Leak potentials 
El_BS = -68.0*mV
E_exc = 0*mV 
E_ahp_BS = -70.0*mV # after hyperpolarization 

# Tau
tau_AMPA_BS = 8.3*ms 
tau_AHP_BS = 2.5*ms 

eqs_BS = Equations('''
                        dV / dt = (-gL_BS * (V - El_BS) - I_syn) / Cm_BS : volt 
                        
                        I_syn = I_AMPA + I_AHP : amp 
                        
                        I_AMPA = g_AMPA_BS * (V - E_exc) * s_AMPA : amp
                        ds_AMPA / dt = - s_AMPA / tau_AMPA_BS : 1 
                        
                        I_AHP = g_AHP_BS * (V - E_ahp_BS) * s_AHP_BS : amp
                        ds_AHP_BS / dt = - s_AHP_BS / tau_AHP_BS : 1
                    ''')

# Defining basket cell neuron group 
basketCell = NeuronGroup(N = numBS, model = eqs_BS, reset = 'V = El_BS',
                    threshold = 'V > theta_BS',
                    method = 'euler')
basketCell.V = El_BS

In [22]:
basketCell

NeuronGroup(clock=Clock(dt=100. * usecond, name='defaultclock'), when=start, order=0, name='neurongroup_3')

### VN

### IO

#  Establishing Connectivity 

#### Parameters 

In [23]:
# Connection probability 
pr_GRGO = 0.0245
#pr_GOGR = 0.025
pr_GOGR = 0.08

#### Granule --> Golgi

In [24]:
# Equations 
eqs_glut_GRGO = '''
                   w_GRGO : 1 
                '''
eqs_pre_glut_GRGO = '''
                       s_AMPA += 1
                    '''

In [25]:
# Forming synapses 
S_GRGO = Synapses(granuleCells, golgi, model = eqs_glut_GRGO, on_pre = eqs_pre_glut_GRGO, method = 'euler') 
S_GRGO.connect(p = pr_GRGO)
S_GRGO.w_GRGO = 0.00004

In [26]:
#visualise_connectivity(S_GRGO)

#### Golgi --> Granule Cells 

In [27]:
# Equations 
eqs_GABA_GOGR = '''
                   w_GOGR : 1 
                '''
eqs_pre_GABA_GOGR = '''
                       s_GABA_1 += 1
                    '''

In [28]:
# Forming synapses 
S_GOGR = Synapses(golgi, granuleCells, model = eqs_GABA_GOGR, on_pre = eqs_pre_GABA_GOGR, method = 'euler') 
#S_GOGR.connect(condition = 'abs(i-j)<2')
S_GOGR.connect(p = pr_GOGR)
S_GOGR.w_GOGR = 10

In [29]:
#visualise_connectivity(S_GOGR)

#### Granule Cell --> Purkinje

In [30]:
# Connecting 9x32 granule cell clusters to purkinje 
index = np.array(*[range(10)], dtype='int')

In [31]:
eqs_glut_GRPKJ = '''
                    w_GRPKJ : 1
                 '''

eqs_pre_glut_GRPKJ = '''
                        s_AMPA += 1 
                     '''


In [32]:
# Connecting 10 granule cells to purkinje 
S_GRPKJ = Synapses(granuleCells, purkinje, model = eqs_glut_GRPKJ, on_pre = eqs_pre_glut_GRPKJ, method = 'euler')
# Iterate over columns 
for k in range(len(purkinje.i)):
    S_GRPKJ.connect(i = (index + (k*10)), j = k)
S_GRPKJ.w_GRPKJ = 0.003


In [33]:
#visualise_connectivity(S_GRPKJ)

#### Granule Cell --> Basket Cell

In [34]:
# Connecting 9x32 granule cell clusters to purkinje 
index = np.array(*[range(10)], dtype='int')

In [35]:
eqs_glut_GRBS = '''
                    w_GRBS : 1
                '''

eqs_pre_glut_GRBS = '''
                        s_AMPA += 1 
                    '''



In [36]:
# Connecting 10 granule cells to basket cell 
S_GRBS = Synapses(granuleCells, basketCell, model = eqs_glut_GRBS, on_pre = eqs_pre_glut_GRBS, method = 'euler')
# Iterate over columns 
for k in range(len(basketCell.i)):
    S_GRBS.connect(i = (index + (k*10)), j = k)
S_GRBS.w_GRBS = 0.003

In [37]:
#visualise_connectivity(S_GRBS)

#### Basket Cell --> Purkinje 

In [38]:
eqs_GABA_BSPKJ = '''
               w_BSPKJ : 1
            '''
eqs_pre_GABA_BSPKJ = '''
                        s_GABA += 1
                     '''

In [39]:
# Connecting purkinje to 3 nearest basket
S_BSPKJ = Synapses(basketCell, purkinje, model = eqs_GABA_BSPKJ, on_pre = eqs_pre_GABA_BSPKJ, method = 'euler')
S_BSPKJ.connect(condition = 'abs(i-j)<2')
S_BSPKJ.w_BSPKJ = 5.3


In [40]:
#visualise_connectivity(S_BSPKJ)

# Running simulation 

In [41]:
def spk_active(monitor, binSize, durationSimulation, numNeurons): 
    # Define bin size 
    bin_size = binSize*ms
    # Define duration of simulation 
    duration = durationSimulation*second
    # Define number of neurons 
    num_Neurons = numNeurons

    # Initialize array to record spike occurancces 
    spkTotal = np.zeros([int(duration/bin_size)])
    
    for current in range(num_Neurons): 
        spk_count, bin_edges = np.histogram(a = np.r_[monitor.t[monitor.i == current]/second],
                                            bins = int(duration/bin_size), range = (0,2))
        # Convert spike count to binary 
        binary = np.where(spk_count > 0, 1, 0)
        # Add to recording array
        spkTotal = spkTotal + binary
    
    return((spkTotal/num_Neurons)*100)

In [42]:
# Setting up monitors

# Granule Cell 
spikeMonitor_GR = SpikeMonitor(granuleCells)
stateMonitor_GR = StateMonitor(granuleCells, 'V', record=True)
prMonitor_GR = PopulationRateMonitor(granuleCells)

# Purkinje 
spikeMonitor_PKJ = SpikeMonitor(purkinje)
stateMonitor_PKJ = StateMonitor(purkinje, 'V', record=True)
prMonitor_PKJ = PopulationRateMonitor(purkinje)

# Basket cells  
spikeMonitor_BS = SpikeMonitor(basketCell)
stateMonitor_BS = StateMonitor(basketCell, 'V', record=True)
prMonitor_BS = PopulationRateMonitor(basketCell)

# Golgi cells  
spikeMonitor_GO = SpikeMonitor(golgi)
stateMonitor_GO = StateMonitor(golgi, 'V', record=True)
prMonitor_GO = PopulationRateMonitor(golgi)

In [43]:
# Granule cell array 
spikeMonitor_Multi_GR = []
spkActive_Multi_GR = []
# Population 
populationRates_GR = []
prMulti_GR_smooth = []
prMulti_GR_t = []

# Purkinje array 
prMulti_PKJ_smooth = []
prMulti_PKJ_t = []

# Basket cell array 
prMulti_BS_smooth = []
prMulti_BS_t = []

# Golgi array 
prMulti_GO_smooth = []
prMulti_GO_t = []

In [44]:
time.time() - start

4.107561111450195

In [45]:
# run the simulation
BrianLogger.log_level_info()
run(1*second)

running brian code generation ...
building genn executable ...
['/hpc/home/kbw29/Code/genn/bin/genn-buildmodel.sh', '-i', '/hpc/home/kbw29/Code/brian2Genn:/hpc/home/kbw29/Code/brian2Genn/Output:/hpc/home/kbw29/Code/brian2Genn/Output/brianlib/randomkit', 'magicnetwork_model.cpp']


RuntimeError: Project compilation failed (Command ['/hpc/home/kbw29/Code/genn/bin/genn-buildmodel.sh', '-i', '/hpc/home/kbw29/Code/brian2Genn:/hpc/home/kbw29/Code/brian2Genn/Output:/hpc/home/kbw29/Code/brian2Genn/Output/brianlib/randomkit', 'magicnetwork_model.cpp'] failed with error code 50).
See the output above (if any) for more details.

#### Benchmarking

In [ ]:
time.time() - start

In [ ]:
# For benchmarking time usage per step ... do not use with multiple runs 
profiling_summary(show = 10)

# Visualization 

In [ ]:
plot_potential(spikeMonitor_GR, stateMonitor_GR, 2)

In [ ]:
plot_spikes(spikeMonitor_GR, stateMonitor_GR, 2)

#### By trial 

In [ ]:
plot_population(prMonitor_GR, spikeMonitor_GR, 'GranuleCells', 100, 10)

In [ ]:
plot_population_Multi(prMulti_GR_smooth[0], prMulti_GR_t[0], spikeMonitor_GR, 'GranuleCells', 100, 10)

In [ ]:
plot_population_Multi(prMulti_GR_smooth[1], prMulti_GR_t[1], spikeMonitor_GR, 'GranuleCells', 100, 10)

#### Individual Granule Cells 

In [ ]:
tempAppend = []
neuronNumber = 0

for i in range(len(spikeMonitor_Multi_GR)):
    tempAppend.extend(pd.array(spikeMonitor_Multi_GR[i][neuronNumber])*1000)

sns.kdeplot(tempAppend, bw_adjust = 0.5)

In [ ]:
tempAppend = []
neuronNumber = 1

for i in range(len(spikeMonitor_Multi_GR)):
    tempAppend.extend(pd.array(spikeMonitor_Multi_GR[i][neuronNumber])*1000)

sns.kdeplot(tempAppend, bw_adjust = 0.2)

In [ ]:
tempAppend = []
neuronNumber = 2

for i in range(len(spikeMonitor_Multi_GR)):
    tempAppend.extend(pd.array(spikeMonitor_Multi_GR[i][neuronNumber])*1000)

sns.kdeplot(tempAppend, bw_adjust = 0.2)

In [ ]:
tempAppend = []
neuronNumber = 43

for i in range(len(spikeMonitor_Multi_GR)):
    tempAppend.extend(pd.array(spikeMonitor_Multi_GR[i][neuronNumber])*1000)

sns.kdeplot(tempAppend, bw_adjust = 0.2)

#### Population Readouts

##### Granule  Cells

In [ ]:
# Plotting average population firing rate across all runs 
average_prMulti = np.mean(prMulti_GR_smooth, axis = 0)

plot_population_Multi_2(average_prMulti, prMulti_GR_t[1], 'GranuleCells')

In [ ]:
active_percent(spikeMonitor_GR, binSize = 10, durationSimulation = 2, numNeurons = numGC, ymax = 20)

In [ ]:
averaged_ActivePercent(spkActive_Multi_GR, spikeMonitor_GR, binSize = 10, durationSimulation = 2, numNeurons = numGC, ymax = 10)

### Purkinje

In [ ]:
plot_potential(spikeMonitor_PKJ, stateMonitor_PKJ, 2)

In [ ]:
plot_spikes(spikeMonitor_PKJ, stateMonitor_PKJ, 2)

In [ ]:
# Plotting average population firing rate across all runs 
average_prMulti_PKJ = np.mean(prMulti_PKJ_smooth, axis = 0)

plot_population_Multi_2(average_prMulti_PKJ, prMulti_PKJ_t[1], 'Purkinje')

## Basket Cell 

In [ ]:
# Plotting average population firing rate across all runs 
average_prMulti_BS = np.mean(prMulti_BS_smooth, axis = 0)

plot_population_Multi_2(average_prMulti_BS, prMulti_BS_t[1], 'Basket Cells')

## Golgi 

In [ ]:
# Plotting average population firing rate across all runs 
average_prMulti_GO = np.mean(prMulti_GO_smooth, axis = 0)

plot_population_Multi_2(average_prMulti_GO, prMulti_GO_t[1], 'Golgi')